<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2018</h1>
<h1 style="font-size: 250%;">Problem Set #1 </h1>
<h1>Issued Tuesday, 1/30/2017; Due Thursday, 2/15/2017</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of	70.71% for each class meeting after the due date. Feel free to get help from others, but the work you submit in should be your own. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded.

In [1]:
# Replace the following string values with the requested information
class Student:
    first = "Mohamed"
    last = "Lansari"
    onyen = "mlans13"
    pid = "730013359"

---
**Problem #1:** 

A. Find the length of the shortest *missing* *k*-mer in the genome of [Salmonella Typhimurium](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.fa) (ignore the second plasmid sequence). Report all missing *k*-mers of that length. 

B. Given that Salmonella's genome is 4857432 base-pairs in length, how does your answer in (a) compare to the *expected* length of the shortest missing *k*-mer? What are the assumptions of you answer, if any? 


In [ ]:
def loadFasta(filename):
    """ Parses a classically formatted and possibly
    compressed FASTA file into a list of headers
    and fragment sequences for each sequence contained"""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'rb')
    else:
        fp = open(filename, 'rb')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

# A)
# Find all of the k-mers in the genome
def findKmers(k, sequence):
    kmerCount = {}
    for i in xrange(len(sequence) - k + 1):
        kmer = sequence[i:i + k]
        kmerCount[kmer] = kmerCount.get(kmer, 0) + 1
    return kmerCount

# Iterate through from k = 1 -> N
def searchKmers():
    k = 1
    headers, seqs = loadFasta("SalmonellaTyphimurium.fa")
    while k < 100:
        count = findKmers(k, seqs[0])
        n = 4 ** k
        num = len(count.keys())
        if n != num:
            break
    


---
**Problem #2:** Modify the <code>kmerPositions()</code> function given in Lecture 2 on slide 20 to allow for one error. Your new function:

     kmersWithError(k, sequence)

should return a Python dictionary whose keys are kmers that appear in either the sequence or its reverse complement, and whose values are a list of positions of a kmer that differs in no more than one base from the key or its reverse complement. Your list should include only the lexigraphically first of each kmer/reverse complement pair. Enter the code for your function in the space provided below.

You can test your code on the following sequence:
<pre>
test = "ggtaattgcctcgcataacgcggtgtgaaaatggattgaagcccgggcggtggattctac"+\
       "tcaactttagccgatggagaaagccccgggatccgggctattaaaaagaagatcttttta"+\
       "tttagagatctgttctattgtgatctcttattaggatcgcgccaggctgtggataacccg"+\
       "gatcctgtaataaagatcaatgcgttggaaaggatcactagctgtgaatgatcggtgatc"+\
       "gtggtccgtataagctgggatcaaaacgggtacttatacacaactcaaaaagtgaacaac"+\
       "ggttattctttggataactaccggttgatccaagctttccaccagatttatccacaatgg"+\
       "atcgcacgatctttacac"
</pre>
But a different sequence will be used for grading.

---
**Problem #3:** Plot the CG skew diagram for [Salmonella Typhimurium](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.fa). Use this to estimate the position of the *OriC*. The true *OriC* position for this genome is between 4082788 and 4084165. How close is your answer to the true *OriC* position?

**Problem #4:** A restriction enzyme is a protein produced by bacteria that cleaves DNA at specific subsequence patterns called *recognition sites*. Bacteria use restriction enzymes to defend against bacterial viruses called bacteriophages. The recognition site of a restriction enzyme is commonly a *reverse palindrome*, meaning the base sequence reads the same backwards and forwards, with a length of 4, 6, or 8 bases. The intervals between recognition sites are called *digestion fragments*.

How many possible reverse palindromic recognition sites of length 8 exist? What fraction of 8-mers are reverse palindromic? Approximately how frequently, in terms of base-pairs, would you expect a given 8-base recogonition site might appear in a genome? How frequently does the recognition site GAATTC appear in chromosome 1 of [Salmonella Typhimurium](http://csbio.unc.edu/mcmillan/Comp555S18/data/SalmonellaTyphimurium.fa)? How many digestion fragments result?

**Problem #5: Programming Problem**

Write a branch-and-bound version of the *MedianStringMotifSearch()* algorithm shown below. You should base your alogrithm on the observation that, if the best alignment of a *prefix* of a candidate motif has a greater Hamming distance than the minimal Hamming distance of the best motif found so far, it could not possibly lead to a better solution.

<pre>
import itertools

def ScanAndScoreMotif(DNA, motif):
    totalDist = 0
    bestAlignment = []
    k = len(motif)
    for seq in DNA:
        minHammingDist = k+1
        for s in xrange(len(seq)-k+1):
            HammingDist = sum([1 for i in xrange(k) if motif[i] != seq[s+i]])
            if (HammingDist < minHammingDist):
                bestS = s
                minHammingDist = HammingDist
        bestAlignment.append(bestS)
        totalDist += minHammingDist
    return bestAlignment, totalDist

def MedianStringMotifSearch(DNA,k):
    """ Consider all possible 4**k motifs"""
    bestAlignment = []
    minHammingDist = k*len(DNA)
    kmer = ''
    for pattern in itertools.product('acgt', repeat=k):
        motif = ''.join(pattern)
        align, dist = ScanAndScoreMotif(DNA, motif)
        if (dist < minHammingDist):
            bestAlignment = [s for s in align]
            minHammingDist = dist
            kmer = motif
    return bestAlignment, minHammingDist, kmer
</pre>

Apply your algoritm to the set of sequences used in lecture:

<pre>
seqApprox = [
    'tagtggtcttttgagtgtagatctgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttggatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagacccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtaaatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattaggtccaaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagattcgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgcagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgta',
    'ttcttacacccttctttagatccaaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtcgatccgaaattcg']
</pre>

Use the %time *magic* function to compare the performance of your method to the one given.

In [ ]:
#Enter your program here

%time BranchAndBoundMedianStringMotifSearch(seqApprox,10)

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set